## Tugas UAS

In [43]:
import numpy as np
import pandas as pd

In [44]:
df = pd.read_csv('tweet_emotions.csv')
display(df.head())

jml_baris_asli = df.shape[0]
print(f'Jumlah baris: {jml_baris_asli}')

,tweet_id,sentiment,content
0,1956967341,empty,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,wants to hang out with friends SOON!
4,1956968416,neutral,@dannycastillo We want to trade with someone w...


Jumlah baris: 40000


#### Preprocessing Data

In [45]:
# Drop twit yang sama
df.drop_duplicates(subset=['content'], inplace=True)

# Cek jumlah data
jml_baris_drop = df.shape[0]
print(f'Jumlah baris: {jml_baris_drop}')
print(f'Jumlah baris duplikasi {jml_baris_asli - jml_baris_drop}')

Jumlah baris: 39827
Jumlah baris duplikasi 173


##### Remove Mention (@) and URL

In [46]:
import re # python regex lib

df = df.copy()

# Membuat kolom baru untuk kebutuhan berbandingan
df['content_clean'] = df['content']

# Membuat fungsi lambda untuk membuat mention, url
rm_rt_url = lambda x: re.sub('(@[A-Za-z0-9\w]+) | (@\w+:) | (\w+:\/\/\S+) | (www.\S+)',' ',x)
rm_punct = lambda x: re.sub('\W', ' ', x)

# Membuat fungsi untuk membuang protocol internet

# Map filter
df['content_clean'] = df.content_clean.map(rm_rt_url).map(rm_punct)
df.head(100)

,tweet_id,sentiment,content,content_clean
0,1956967341,empty,@tiffanylue i know i was listenin to bad habi...,i know i was listenin to bad habit earlier a...
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...,Layin n bed with a headache ughhhh waitin o...
2,1956967696,sadness,Funeral ceremony...gloomy friday...,Funeral ceremony gloomy friday
3,1956967789,enthusiasm,wants to hang out with friends SOON!,wants to hang out with friends SOON
4,1956968416,neutral,@dannycastillo We want to trade with someone w...,We want to trade with someone who has Houston...
...,...,...,...,...
95,1956989514,sadness,@sweeetnspicy hiii im on my ipod...i cant fall...,hiii im on my ipod i cant fall asleep
96,1956989526,sadness,dont wanna work 11-830 tomorrow but i get paid,dont wanna work 11 830 tomorrow but i get paid
97,1956989560,sadness,feels sad coz i wasnt able to play with the gu...,feels sad coz i wasnt able to play with the gu...
98,1956989561,neutral,PrinceCharming,PrinceCharming


##### Case Folding

In [47]:
df['content_clean'] = df.content_clean.str.lower()
df.tail(10)

,tweet_id,sentiment,content,content_clean
39990,1753918829,neutral,@shonali I think the lesson of the day is not ...,i think the lesson of the day is not to have ...
39991,1753918846,neutral,@lovelylisaj can you give me the link for the ...,can you give me the link for the kimba diarie...
39992,1753918881,neutral,@jasimmo Ooo showing of your French skills!! l...,ooo showing of your french skills lol thing...
39993,1753918892,neutral,"@sendsome2me haha, yeah. Twitter has many uses...",haha yeah twitter has many uses for me it ...
39994,1753918900,happiness,Succesfully following Tayla!!,succesfully following tayla
39995,1753918954,neutral,@JohnLloydTaylor,johnlloydtaylor
39996,1753919001,love,Happy Mothers Day All my love,happy mothers day all my love
39997,1753919005,love,Happy Mother's Day to all the mommies out ther...,happy mother s day to all the mommies out ther...
39998,1753919043,happiness,@niariley WASSUP BEAUTIFUL!!! FOLLOW ME!! PEE...,wassup beautiful follow me peep out my ...
39999,1753919049,love,@mopedronin bullet train from tokyo the gf ...,bullet train from tokyo the gf and i have ...


##### Tokenizing

In [48]:
from nltk.tokenize import TweetTokenizer
df_stem = df.copy()

tweet_token = TweetTokenizer()
df_stem['content_token'] = df_stem['content_clean'].apply(tweet_token.tokenize)

df_stem.head()


,tweet_id,sentiment,content,content_clean,content_token
0,1956967341,empty,@tiffanylue i know i was listenin to bad habi...,i know i was listenin to bad habit earlier a...,"[i, know, i, was, listenin, to, bad, habit, ea..."
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...,layin n bed with a headache ughhhh waitin o...,"[layin, n, bed, with, a, headache, ughhhh, wai..."
2,1956967696,sadness,Funeral ceremony...gloomy friday...,funeral ceremony gloomy friday,"[funeral, ceremony, gloomy, friday]"
3,1956967789,enthusiasm,wants to hang out with friends SOON!,wants to hang out with friends soon,"[wants, to, hang, out, with, friends, soon]"
4,1956968416,neutral,@dannycastillo We want to trade with someone w...,we want to trade with someone who has houston...,"[we, want, to, trade, with, someone, who, has,..."


##### Stemming

In [49]:
# Stemming
from nltk.stem import SnowballStemmer

stemmer = SnowballStemmer("english")

def stemming(text):
    stem_text = [stemmer.stem(word) for word in text]
    return stem_text

df_stem['content_stem'] = df_stem['content_token'].apply(lambda x: stemming(x))

df_stem.head()

,tweet_id,sentiment,content,content_clean,content_token,content_stem
0,1956967341,empty,@tiffanylue i know i was listenin to bad habi...,i know i was listenin to bad habit earlier a...,"[i, know, i, was, listenin, to, bad, habit, ea...","[i, know, i, was, listenin, to, bad, habit, ea..."
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...,layin n bed with a headache ughhhh waitin o...,"[layin, n, bed, with, a, headache, ughhhh, wai...","[layin, n, bed, with, a, headach, ughhhh, wait..."
2,1956967696,sadness,Funeral ceremony...gloomy friday...,funeral ceremony gloomy friday,"[funeral, ceremony, gloomy, friday]","[funer, ceremoni, gloomi, friday]"
3,1956967789,enthusiasm,wants to hang out with friends SOON!,wants to hang out with friends soon,"[wants, to, hang, out, with, friends, soon]","[want, to, hang, out, with, friend, soon]"
4,1956968416,neutral,@dannycastillo We want to trade with someone w...,we want to trade with someone who has houston...,"[we, want, to, trade, with, someone, who, has,...","[we, want, to, trade, with, someon, who, has, ..."


##### Clustering

In [51]:
# Clustering Pengelompokan sentiment  yang bernilai positif, negatif, dan netral
from sklearn.cluster import KMeans

# Membuat fungsi untuk mengelompokkan sentiment
def sentiment_cluster(sentiment):
    if sentiment == 'empty':
        return 'netral'
    elif sentiment == 'sadness':
        return 'negatif'
    elif sentiment == 'enthusiasm':
        return 'positif'
    elif sentiment == 'neutral':
        return 'netral'
    elif sentiment == 'worry':
        return 'negatif'
    elif sentiment == 'surprise':
        return 'positif'
    elif sentiment == 'love':
        return 'positif'
    elif sentiment == 'fun':
        return 'positif'
    elif sentiment == 'hate':
        return 'negatif'
    elif sentiment == 'happiness':
        return 'positif'
    elif sentiment == 'boredom':
        return 'negatif'
    elif sentiment == 'relief':
        return 'positif'
    elif sentiment == 'anger':
        return 'negatif'

# Membuat kolom baru untuk sentiment
df_stem['sentiment_cluster'] = df_stem['sentiment'].apply(sentiment_cluster)

df_stem.head()

,tweet_id,sentiment,content,content_clean,content_token,content_stem,sentiment_cluster
0,1956967341,empty,@tiffanylue i know i was listenin to bad habi...,i know i was listenin to bad habit earlier a...,"[i, know, i, was, listenin, to, bad, habit, ea...","[i, know, i, was, listenin, to, bad, habit, ea...",netral
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...,layin n bed with a headache ughhhh waitin o...,"[layin, n, bed, with, a, headache, ughhhh, wai...","[layin, n, bed, with, a, headach, ughhhh, wait...",negatif
2,1956967696,sadness,Funeral ceremony...gloomy friday...,funeral ceremony gloomy friday,"[funeral, ceremony, gloomy, friday]","[funer, ceremoni, gloomi, friday]",negatif
3,1956967789,enthusiasm,wants to hang out with friends SOON!,wants to hang out with friends soon,"[wants, to, hang, out, with, friends, soon]","[want, to, hang, out, with, friend, soon]",positif
4,1956968416,neutral,@dannycastillo We want to trade with someone w...,we want to trade with someone who has houston...,"[we, want, to, trade, with, someone, who, has,...","[we, want, to, trade, with, someon, who, has, ...",netral
